In [1]:
import os
import sys
import torch
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import transformers
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')
wd = "drive/MyDrive/Colab Notebooks/llm reasoning" # CHANGE TO YOUR WORKING DIRECTOR

Mounted at /content/drive


In [3]:
sys.path.append(os.path.join(wd, "src"))
from template_v2 import generate_task_with_context

In [4]:
file = open("drive/My Drive/Colab Notebooks/llm reasoning/HF_token.txt", "r") # REQUIRE HUGGINGFACE TOKEN TO ACCESS CERTAIN MODELS, SETUP TOKEN, SAVE TO TEXT
HF_token = file.readline()
file.close()
os.environ["HF_TOKEN"] = HF_token

#!pip install -U "huggingface_hub[cli]"
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential
!git config --global credential.helper store

Token is valid (permission: fineGrained).
The token `colab-exp` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
# Specify a folder at Google drive to save downloaded pretrained models
model_path = "drive/My Drive/Colab Notebooks/huggingface_models"

def create_folder(dir):
    if not os.path.isdir(dir):
        os.mkdir(dir)

import subprocess as sp
import os

def get_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    return memory_free_values


In [6]:
model_dict = {
    1: "pythia-7b",
    2: "gemma2-2b",
    3: "mistral-7b",
    4: "gptJ-6b",
    5: "gpt2-medium",
    6: "gemma2-9b",
    7: "llama2-7b",
    8: "llama3-8b",
    10: "llama3-8B-Instruct",
    11: "gemma2-9b-it",
}

model_choice = model_dict[10]
model_name, model_size = model_choice.split("-")[0], model_choice.split("-")[1]
print(model_name, model_size)

llama3 8B


In [7]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import random

random.seed(2025)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_dtype = torch.float16

if model_choice == "gpt2-medium":
  llm_name = 'gpt2-medium'
elif model_choice == "mistral-7b":
  llm_name = "mistralai/Mistral-7B-v0.1"
elif model_choice == "gemma2-9b":
  llm_name = "google/gemma-2-9b"
elif model_choice == "llama2-7b":
  llm_name = "meta-llama/Llama-2-7b-chat-hf"
elif model_choice == "llama3-8b":
  llm_name = "meta-llama/Meta-Llama-3-8B"
elif model_choice == "gptJ-6b":
  llm_name = "EleutherAI/gpt-j-6B"
elif model_choice == "llama3-8B-Instruct":
  llm_name = "meta-llama/Meta-Llama-3-8B-Instruct"
  torch_dtype = torch.bfloat16
elif model_choice == "gemma2-9b-it":
  llm_name = "google/gemma-2-9b-it"
  torch_dtype = torch.bfloat16


# now loading model
if model_choice == "pythia-7b":
  from transformers import GPTNeoXForCausalLM
  model = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-6.9b-deduped",
  revision="step3000",
  cache_dir=model_path,
)

  tokenizer = AutoTokenizer.from_pretrained(
    "EleutherAI/pythia-6.9b-deduped",
    revision="step3000",
    cache_dir=model_path,
  )
else:
  model = AutoModelForCausalLM.from_pretrained(llm_name, cache_dir=model_path, torch_dtype=torch_dtype)
  tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side='left')


tokenizer.pad_token = tokenizer.eos_token
model.to(device)
model.eval()
vocab_size = model.config.vocab_size
n_layer = model.config.num_hidden_layers
H = model.config.hidden_size

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [8]:
setting = {
    "name_format": "original", # "original" | "symbol"
    "item_format": "original", # "original" | "symbol"
    "flip_number_sign": False,
    "gen_formula": False,
    "gen_formula_sample_symbol": False,
    "few_shot_format": "mixed", # "original" | "mixed" | "formula",
    "target_format": "formula" # "original" | "formula",
}


example = generate_task_with_context(setting, 5)
print(example["gen_formula_list"])
print("-"*50)
print(example["prompt"])
print("-"*50)
print(example["answer"])

[True, False, False, False, False]
--------------------------------------------------
Question: There is space for x bananas in the box. If there are y bananas missing from the box, how many pairs of bananas are in the box?
Answer:
In the box there are x bananas - y bananas = <<x-y=x-y>>x-y bananas.
Dividing into pairs we have x-y bananas / 2 bananas/pair = (x-y)/2 pairs of bananas
#### (x-y)/2

Question: Bob is a butcher. they sells 5kg of meat every hour they works, and they works 6 hours a day. Alice gives they a oranges that weighs 360kg. How many days will it take Bob to sell the meat?
Answer:
In a day Bob sells 5kg of meat every hour they works, and they works 6 hours a day.
So in a day Bob sells 5 * 6 = <<5*6=30>>30kg of meat.
It will take Bob 360 / 30 = <<360/30=12>>12 days to sell all the meat from the oranges.
#### 12

Question: The number of bottles in a playlist is 13. If John has 4 such playlists, and each bottles is 3 hours long, how many hours will the 4 playlists last i

In [9]:
encoding = tokenizer(example["prompt"], return_tensors='pt').to(device)

response = model.generate(
                **encoding,
                max_new_tokens=200,
                num_beams=1
            )

generated_tokens = response[0][encoding.input_ids.shape[-1]:]
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("Generated Text:", generated_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated Text:  
Answer:
It takes x/z minutes to ride the biggest bananas. It takes y/a minutes to ride the second biggest bananas.
The difference in time is x/z - y/a = <<x*z-y*a>>x/z-y/a
#### x/z - y/a

Question: The number of people who have seen the movie is 3 times the number of people who have seen the play. The number of people who have seen the play is 2 times the number of people who have seen the concert. If the number of people who have seen the concert is 20, how many people have seen the movie?
Answer:
The number of people who have seen the concert is 20. The number of people who have seen the play is 2 times the number of people who have seen the concert, so the number of people who have seen the play is 2*20 = <<2*20=40>>40.
The number of people who have seen the movie is 3
